In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import numpy as np
import os
import torch

import torch.nn as nn
import torch.optim as optim

from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import EfficientNet

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
CONTINUE_TRAINING_DIR = 'notebooks/efficientnet/models/'
MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
DATA_DIR = 'E:/Datasets/cor-splits/sgkf-8-1-1-2000'
START_EPOCH = 16 # Change it if you're continuing an existing run
N_EPOCHS = 50
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0
MIN_EPOCH = 10

In [4]:
working_folder = 'notebooks/efficientnet'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [5]:
# # efficientnet_b0
# mean = np.array([0.485, 0.456, 0.406])
# std = np.array([0.229, 0.224, 0.225])

# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomHorizontalFlip(),
#         transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(), # When converting to tensor, pytorch automatically rescales to [0,1]
#         transforms.Normalize(mean, std)
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std)
#     ]),
# }

In [6]:
# model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

# for param in model.parameters(): # Turning parameters not trainable
#     param.requires_grad = True

# # A newly defined layer is created with requires_grad=True by default
# model.classifier = nn.Sequential(
#     nn.Dropout(p=0.2, inplace=True),
#     nn.Linear(in_features=1280, out_features=2)
#     )

# if CONTINUE_TRAINING:
#     output_folder = MODEL_WEIGHTS_PATH
#     model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
# else:
#     if not os.path.exists(output_folder): 
#         os.makedirs(output_folder)

In [7]:
eff_net = EfficientNet()

if CONTINUE_TRAINING:
    model = eff_net.load(MODEL_WEIGHTS_PATH)
else:
    model = eff_net.load()

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
             
model = model.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()

# optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=LR)
optimizer = optim.Adamax(model.parameters(), lr=LR)

# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA, last_epoch=START_EPOCH)
es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)

In [9]:
dataset = PytorchDataset(DATA_DIR, eff_net.data_transforms, BATCH_SIZE)
trainer = PytorchTraining(device, dataset, output_folder)
model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/50
----------
train Loss: 0.1704 Acc: 0.9312
val Loss: 0.6749 Acc: 0.7770
Epoch duration: 0 m 49s
Learning Rate = 0.001


Epoch 2/50
----------
train Loss: 0.0438 Acc: 0.9831
val Loss: 0.4840 Acc: 0.8474
Epoch duration: 0 m 50s
Learning Rate = 0.001


Epoch 3/50
----------
train Loss: 0.0313 Acc: 0.9887
val Loss: 0.5286 Acc: 0.8239
Epoch duration: 0 m 48s
Learning Rate = 0.001


Epoch 4/50
----------
train Loss: 0.0197 Acc: 0.9932
val Loss: 0.7966 Acc: 0.7864
Epoch duration: 0 m 58s
Learning Rate = 0.001


Epoch 5/50
----------
train Loss: 0.0170 Acc: 0.9941
val Loss: 0.7025 Acc: 0.8005
Epoch duration: 0 m 55s
Learning Rate = 0.001


Epoch 6/50
----------
train Loss: 0.0148 Acc: 0.9947
val Loss: 0.5587 Acc: 0.8146
Epoch duration: 0 m 57s
Learning Rate = 0.001


Epoch 7/50
----------
train Loss: 0.0116 Acc: 0.9956
val Loss: 0.5192 Acc: 0.8404
Epoch duration: 0 m 53s
Learning Rate = 0.001


Epoch 8/50
----------
train Loss: 0.0156 Acc: 0.9935
val Loss: 0.6153 Acc: 0.8052
Epoch du